<a href="https://colab.research.google.com/github/atm1504/mongodb-details/blob/master/update_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Update
Script to learn updating of data, mainly using python code, instead of Aggregate FrameWork.

In [1]:
!pip3 install pymongo[srv]

     |████████████████████████████████| 194kB 2.7MB/s 


In [11]:
import pymongo
from pymongo import MongoClient, UpdateOne
import pprint
from datetime import datetime
import re
from IPython.display import clear_output

In [2]:
# Replace XXXX with your connection URI from the Atlas UI
client = MongoClient(XXXX)

In [12]:
## Copy data into a new collection from milf, so that manin data don't gets hampered
pipeline=[{
    "$limit":2200
},{ '$match': {} },
{
    "$out":"movies"
}]
clear_output()
pprint.pprint(list(client.mongo.milf.aggregate(pipeline)))

[]


## Update one-by-one
In this section we parse through the movies one by one, and updatet the values. We maintain to different type of variable to decide which field to drop and which field to add or update.

In [10]:
runtime_pat = re.compile(r'([0-9]+) min')

for movie in client.mongo.movies.find({}).limit(10):

    fields_to_set = {}
    fields_to_unset = {}

    for k,v in movie.copy().items():
        if v == "" or v == [""]:
            del movie[k]
            fields_to_unset[k] = ""

    if 'director' in movie:
        fields_to_unset['director'] = ""
        fields_to_set['directors'] = movie['director'].split(", ")
    if 'cast' in movie:
        fields_to_set['cast'] = movie['cast'].split(", ")
    if 'writer' in movie:
        fields_to_unset['writer'] = ""
        fields_to_set['writers'] = movie['writer'].split(", ")
    if 'genre' in movie:
        fields_to_unset['genre'] = ""
        fields_to_set['genres'] = movie['genre'].split(", ")
    if 'language' in movie:
        fields_to_unset['language'] = ""
        fields_to_set['languages'] = movie['language'].split(", ")
    if 'country' in movie:
        fields_to_unset['country'] = ""
        fields_to_set['countries'] = movie['country'].split(", ")
        
    if 'fullplot' in movie:
        fields_to_unset['fullplot'] = ""
        fields_to_set['fullPlot'] = movie['fullplot']
    if 'rating' in movie:
        fields_to_unset['rating'] = ""
        fields_to_set['rated'] = movie['rating']

    imdb = {}
    if 'imdbID' in movie:
        fields_to_unset['imdbID'] = ""
        imdb['id'] = movie['imdbID']
    if 'imdbRating' in movie:
        fields_to_unset['imdbRating'] = ""
        imdb['rating'] = movie['imdbRating']
    if 'imdbVotes' in movie:
        fields_to_unset['imdbVotes'] = ""
        imdb['votes'] = movie['imdbVotes']
    if imdb:
        fields_to_set['imdb'] = imdb
        
    if 'released' in movie:
        fields_to_set['released'] = datetime.strptime(movie['released'],
                                                      "%Y-%m-%d")
    if 'lastUpdated' in movie:
        fields_to_set['lastUpdated'] = datetime.strptime(movie['lastUpdated'][0:19],
                                                         "%Y-%m-%d %H:%M:%S")

    if 'runtime' in movie:
        m = runtime_pat.match(movie['runtime']) 
        if m:
            fields_to_set['runtime'] = int(m.group(1))

    update_doc = {}
    if fields_to_set:
        update_doc['$set'] = fields_to_set
    if fields_to_unset:
        update_doc['$unset'] = fields_to_unset
    pprint.pprint(update_doc)

    client.mongo.movies.update_one({'_id': movie['_id']}, update_doc)


{'$set': {'cast': ['Carmencita'],
          'countries': ['USA'],
          'directors': ['William K.L. Dickson'],
          'fullPlot': 'Performing on what looks like a small wooden stage, '
                      'wearing a dress with a hoop skirt and white high-heeled '
                      'pumps, Carmencita does a dance with kicks and twirls, a '
                      'smile always on her face.',
          'genres': ['Documentary', 'Short'],
          'imdb': {'id': '1', 'rating': '5.9', 'votes': '1032'},
          'rated': 'NOT RATED',
          'runtime': 1},
 '$unset': {'awards': '',
            'country': '',
            'director': '',
            'fullplot': '',
            'genre': '',
            'imdbID': '',
            'imdbRating': '',
            'imdbVotes': '',
            'language': '',
            'metacritic': '',
            'rating': '',
            'released': '',
            'writer': ''}}
{'$set': {'cast': ['Fred Ott'],
          'countries': ['USA'],
     

## Bulk Update

Before testing this, delete the old movies collection. Copy the items of milf into movies, then execute it.

In [13]:
runtime_pat = re.compile(r'([0-9]+) min')

batch_size = 1000
updates = []
count = 0
for movie in client.mongo.movies.find({}):

    fields_to_set = {}
    fields_to_unset = {}

    for k,v in movie.copy().items():
        if v == "" or v == [""]:
            del movie[k]
            fields_to_unset[k] = ""

    if 'director' in movie:
        fields_to_unset['director'] = ""
        fields_to_set['directors'] = movie['director'].split(", ")
    if 'cast' in movie:
        fields_to_set['cast'] = movie['cast'].split(", ")
    if 'writer' in movie:
        fields_to_unset['writer'] = ""
        fields_to_set['writers'] = movie['writer'].split(", ")
    if 'genre' in movie:
        fields_to_unset['genre'] = ""
        fields_to_set['genres'] = movie['genre'].split(", ")
    if 'language' in movie:
        fields_to_unset['language'] = ""
        fields_to_set['languages'] = movie['language'].split(", ")
    if 'country' in movie:
        fields_to_unset['country'] = ""
        fields_to_set['countries'] = movie['country'].split(", ")
        
    if 'fullplot' in movie:
        fields_to_unset['fullplot'] = ""
        fields_to_set['fullPlot'] = movie['fullplot']
    if 'rating' in movie:
        fields_to_unset['rating'] = ""
        fields_to_set['rated'] = movie['rating']

    imdb = {}
    if 'imdbID' in movie:
        fields_to_unset['imdbID'] = ""
        imdb['id'] = movie['imdbID']
    if 'imdbRating' in movie:
        fields_to_unset['imdbRating'] = ""
        imdb['rating'] = movie['imdbRating']
    if 'imdbVotes' in movie:
        fields_to_unset['imdbVotes'] = ""
        imdb['votes'] = movie['imdbVotes']
    if imdb:
        fields_to_set['imdb'] = imdb
        
    if 'released' in movie:
        fields_to_set['released'] = datetime.strptime(movie['released'],
                                                      "%Y-%m-%d")
    if 'lastUpdated' in movie:
        fields_to_set['lastUpdated'] = datetime.strptime(movie['lastUpdated'][0:19],
                                                         "%Y-%m-%d %H:%M:%S")

    if 'runtime' in movie:
        m = runtime_pat.match(movie['runtime']) 
        if m:
            fields_to_set['runtime'] = int(m.group(1))

    update_doc = {}
    if fields_to_set:
        update_doc['$set'] = fields_to_set
    if fields_to_unset:
        update_doc['$unset'] = fields_to_unset

    updates.append(UpdateOne({'_id': movie['_id']}, update_doc))

    count += 1
    if count == batch_size:
        client.mongo.movies.bulk_write(updates)
        updates = []
        count = 0

if updates:         
    client.mongo.movies.bulk_write(updates)

In [14]:
# Seeing data after updating it.
for movie in client.mongo.movies.find({}).limit(10):
  pprint.pprint(movie)

{'_id': ObjectId('5ef8953205d42488e36c6b75'),
 'cast': ['Carmencita'],
 'countries': ['USA'],
 'directors': ['William K.L. Dickson'],
 'fullPlot': 'Performing on what looks like a small wooden stage, wearing a '
             'dress with a hoop skirt and white high-heeled pumps, Carmencita '
             'does a dance with kicks and twirls, a smile always on her face.',
 'genres': ['Documentary', 'Short'],
 'imdb': {'id': '1', 'rating': '5.9', 'votes': '1032'},
 'lastupdated': '2015-08-26 00:03:45.040000000',
 'plot': 'Performing on what looks like a small wooden stage, wearing a dress '
         'with a hoop skirt and white high-heeled pumps, Carmencita does a '
         'dance with kicks and twirls, a smile always on her face.',
 'poster': 'https://m.media-amazon.com/images/M/MV5BMjAzNDEwMzk3OV5BMl5BanBnXkFtZTcwOTk4OTM5Ng@@._V1_SX300.jpg',
 'rated': 'NOT RATED',
 'runtime': 1,
 'title': 'Carmencita',
 'type': 'movie',
 'year': '1894'}
{'_id': ObjectId('5ef8953205d42488e36c6b78'),
 'ca